# Лабораторная работа №8: Проведение исследований с моделями обнаружения и распознавания объектов (Ultralytics YOLO)

## 1. Выбор начальных условий

### 1.a. Выбор набора данных и обоснование

Для данной лабораторной работы был выбран набор данных **COCO128**. Это небольшой, но репрезентативный поднабор данных из известного датасета COCO (Common Objects in Context).

**Обоснование выбора:**
1.  **Доступность и простота использования:** COCO128 поставляется вместе с Ultralytics YOLO, что упрощает его загрузку и использование без необходимости ручной подготовки данных.
2.  **Реальная практическая задача:** Обнаружение объектов на изображениях является фундаментальной задачей компьютерного зрения с множеством практических применений:
    *   **Автономное вождение:** Обнаружение пешеходов, автомобилей, дорожных знаков.
    *   **Видеонаблюдение и безопасность:** Идентификация подозрительных объектов или действий.
    *   **Розничная торговля:** Анализ поведения покупателей, управление запасами (например, подсчет товаров на полках).
    *   **Медицина:** Обнаружение аномалий на медицинских снимках (например, опухолей).
    *   **Сельское хозяйство:** Мониторинг состояния посевов, подсчет скота.
3.  **Разнообразие классов:** Несмотря на небольшой размер, COCO128 содержит изображения с объектами различных классов (80 классов из оригинального COCO), что позволяет обучать и тестировать модели на разнообразных сценариях.
4.  **Подходит для демонстрации:** Малый размер датасета позволяет быстро проводить эксперименты, обучать модели и получать результаты в рамках лабораторной работы, не требуя значительных вычислительных ресурсов или времени.
5.  **Стандартный формат аннотаций:** Данные представлены в формате YOLO, который является стандартом де-факто для многих моделей обнаружения объектов.

Таким образом, COCO128 представляет собой удобный и релевантный датасет для изучения и экспериментирования с моделями обнаружения объектов семейства YOLO.

### 1.b. Выбор метрик качества и обоснование

Для оценки качества моделей обнаружения объектов будут использоваться следующие метрики:

1.  **Precision (Точность):**
    *   Формула: `Precision = TP / (TP + FP)`_
    *   Описание: Доля правильно идентифицированных положительных объектов среди всех объектов, которые модель классифицировала как положительные. Отвечает на вопрос: "Из всех объектов, которые модель назвала 'объектом X', сколько действительно являются 'объектом X'?"
    *   Обоснование: Важна, когда цена ложноположительных срабатываний высока (например, в системах безопасности, где ложная тревога может привести к ненужным действиям).

2.  **Recall (Полнота):**
    *   Формула: `Recall = TP / (TP + FN)`
    *   Описание: Доля правильно идентифицированных положительных объектов среди всех реально существующих положительных объектов. Отвечает на вопрос: "Сколько 'объектов X' из всех реально существующих 'объектов X' модель смогла найти?"
    *   Обоснование: Важна, когда цена ложноотрицательных срабатываний высока (например, в медицине, где пропуск заболевания может иметь серьезные последствия, или при поиске дефектов).

3.  **mAP (mean Average Precision - средняя точность):**
    *   **AP (Average Precision):** Площадь под кривой Precision-Recall для одного класса. AP вычисляется для каждого класса объектов отдельно.
    *   **mAP@0.5 (или mAP50):** Среднее значение AP по всем классам, вычисленное при пороге IoU (Intersection over Union) равном 0.5. IoU измеряет степень пересечения между предсказанной ограничивающей рамкой и истинной рамкой. Порог 0.5 означает, что предсказание считается верным (TP), если IoU > 0.5.
    *   **mAP@0.5:0.95 (или mAP):** Среднее значение AP по всем классам, вычисленное для различных порогов IoU (от 0.5 до 0.95 с шагом 0.05) и затем усредненное. Эта метрика более строгая и дает более комплексную оценку качества модели.

    *   Обоснование: mAP является основной и наиболее комплексной метрикой для задач обнаружения объектов. Она учитывает как точность, так и полноту модели для каждого класса, а также качество локализации объектов (через IoU). mAP@0.5:0.95 дает наиболее объективную оценку общей производительности модели в различных условиях.

**Выбор данных метрик обусловлен их широким распространением в области обнаружения объектов, способностью комплексно оценивать как классификационную, так и локализационную точность модели.**



In [1]:
# Установка Ultralytics
!pip install ultralytics -q
!pip install roboflow -q # На всякий случай, если понадобится для другого датасета


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [2]:
import ultralytics
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import yaml # Для работы с YAML файлами датасета

# Проверка установки
ultralytics.checks()

# Глобальные переменные для хранения результатов
results_history = {}


Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/112.6 GB disk)


## 2. Создание бейзлайна и оценка качества

На этом этапе мы обучим одну из стандартных моделей YOLO (например, YOLOv8n - nano, самую маленькую и быструю) на нашем наборе данных `coco128.yaml` с базовыми настройками. Это будет наш бейзлайн.

### 2.a. Обучение "коробочной" модели YOLO из Ultralytics
Будем использовать YOLOv8n (nano) как самую быструю для демонстрации
epochs можно поставить поменьше для скорости, например 10-25 для лабораторной
Для реальных задач нужно больше, например, 100-300.
imgsz - размер изображений, на которых обучается модель


In [3]:
 # Задаем параметры
dataset_yaml = 'coco128.yaml' # Стандартный датасет Ultralytics, загрузится автоматически
baseline_model_name = 'yolov8n.pt' # Используем предварительно обученную модель yolov8n
baseline_epochs = 25 # Уменьшим для скорости выполнения лабораторной
baseline_imgsz = 640
baseline_project_name = 'coco128_baseline'
baseline_experiment_name = 'yolov8n_baseline_run'

print(f"Обучение бейзлайн модели: {baseline_model_name}")
print(f"Датасет: {dataset_yaml}")
print(f"Количество эпох: {baseline_epochs}")
print(f"Размер изображения: {baseline_imgsz}")

# Создаем модель
model_baseline = YOLO(baseline_model_name)

# Обучение модели
# device=0 означает использование первого доступного GPU, если есть, иначе CPU.
# В Colab это обычно GPU Tesla T4, K80 и т.д.
try:
    history_baseline = model_baseline.train(
        data=dataset_yaml,
        epochs=baseline_epochs,
        imgsz=baseline_imgsz,
        project=baseline_project_name,
        name=baseline_experiment_name,
        exist_ok=True, # Перезаписывать, если эксперимент уже существует
        patience=5 # Ранняя остановка, если нет улучшений в течение 5 эпох
    )
    print("Обучение бейзлайн модели завершено.")
    baseline_weights_path = os.path.join(baseline_project_name, baseline_experiment_name, 'weights', 'best.pt')
    print(f"Лучшие веса сохранены в: {baseline_weights_path}")

except Exception as e:
    print(f"Ошибка во время обучения бейзлайн модели: {e}")
    # В случае ошибки (например, нет GPU или битая установка)
    # можно попробовать device='cpu' или уменьшить batch size (batch=-1 для автоподбора)
    # history_baseline = model_baseline.train(data=dataset_yaml, epochs=baseline_epochs, imgsz=baseline_imgsz, device='cpu', batch=8) # Пример для CPU
    history_baseline = None # или какое-то значение по умолчанию/обработка
    baseline_weights_path = None



Обучение бейзлайн модели: yolov8n.pt
Датасет: coco128.yaml
Количество эпох: 25
Размер изображения: 640


100%|██████████| 6.25M/6.25M [00:00<00:00, 107MB/s]

Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_baseline_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=coco128_baseline, rect=False, resume=False, retina_masks=

100%|██████████| 6.66M/6.66M [00:00<00:00, 108MB/s]
Unzipping /content/datasets/coco128.zip to /content/datasets/coco128...: 100%|██████████| 263/263 [00:00<00:00, 3420.39file/s]

...

      25/25      3.63G      1.049     0.9869      1.117        133        640: 100%|██████████| 8/8 [00:01<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.98it/s]

                   all        128        929      0.714      0.742      0.785      0.602



25 epochs completed in 0.025 hours.
Optimizer stripped from coco128_baseline/yolov8n_baseline_run/weights/last.pt, 6.5MB
Optimizer stripped from coco128_baseline/yolov8n_baseline_run/weights/best.pt, 6.5MB

Validating coco128_baseline/yolov8n_baseline_run/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.65it/s]


                   all        128        929      0.724       0.74      0.786      0.602
                person         61        254       0.87      0.686      0.808      0.601
               bicycle          3          6      0.726      0.333      0.512      0.401
                   car         12         46      0.621      0.261      0.412      0.236
            motorcycle          4          5      0.768        0.8      0.938      0.787
              airplane          5          6      0.781          1      0.948      0.881
                   bus          5          7      0.786      0.714      0.723      0.652
                 train          3          3      0.575          1      0.995      0.886
                 truck          5         12      0.822        0.5      0.592      0.413
                  boat          2          6      0.636      0.667      0.636      0.396
         traffic light          4         14      0.542      0.143      0.205      0.151
             stop sig

### 2.b. Оценка качества модели (бейзлайн) по выбранным метрикам
Оценка будет проводиться на валидационном наборе данных, указанном в coco128.yaml


In [4]:
 if baseline_weights_path and os.path.exists(baseline_weights_path):
    print(f"Оценка бейзлайн модели с весами: {baseline_weights_path}")
    # Загружаем лучшую модель, полученную после обучения
    model_eval_baseline = YOLO(baseline_weights_path)

    # Проводим оценку (валидацию)
    # split='val' по умолчанию, verbose=True для вывода метрик по классам
    metrics_baseline = model_eval_baseline.val(
        data=dataset_yaml,
        imgsz=baseline_imgsz,
        split='val',
        project=baseline_project_name,
        name=baseline_experiment_name + "_eval",
        exist_ok=True
    )
    print("\nРезультаты оценки бейзлайн модели:")
    if metrics_baseline:
      print(f"  Precision(B): {metrics_baseline.box.mp:.4f}") # mP средняя точность по всем классам
      print(f"  Recall(B): {metrics_baseline.box.mr:.4f}")    # mR средняя полнота по всем классам
      print(f"  mAP50(B): {metrics_baseline.box.map50:.4f}")
      print(f"  mAP50-95(B): {metrics_baseline.box.map:.4f}") # Основная метрика mAP

      results_history['baseline'] = {
          'model': baseline_model_name.split('.')[0],
          'weights': baseline_weights_path,
          'precision': metrics_baseline.box.mp,
          'recall': metrics_baseline.box.mr,
          'mAP50': metrics_baseline.box.map50,
          'mAP50-95': metrics_baseline.box.map
      }
    else:
        print("Не удалось получить метрики для бейзлайн модели.")
        results_history['baseline'] = {
          'model': baseline_model_name.split('.')[0],
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
        }

    # Показать примеры предсказаний
    # val_results_path = os.path.join(baseline_project_name, baseline_experiment_name + "_eval")
    # print(f"Результаты валидации (изображения с рамками) сохранены в: {val_results_path}")
    # # Показать первые несколько изображений с предсказаниями (если они были сохранены)
    # pred_images = [os.path.join(val_results_path, img) for img in os.listdir(val_results_path) if img.endswith(('.png', '.jpg', '.jpeg'))][:3]
    # for img_path in pred_images:
    #     plt.figure(figsize=(8,8))
    #     plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    #     plt.title(os.path.basename(img_path))
    #     plt.axis('off')
    #     plt.show()

else:
    print("Файл с весами бейзлайн модели не найден. Оценка невозможна.")
    results_history['baseline'] = {
          'model': baseline_model_name.split('.')[0],
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
    }


Оценка бейзлайн модели с весами: coco128_baseline/yolov8n_baseline_run/weights/best.pt
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1041.3±367.7 MB/s, size: 53.4 KB)


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.95it/s]


                   all        128        929      0.763      0.728      0.786      0.605
                person         61        254      0.899      0.669      0.811      0.601
               bicycle          3          6      0.754      0.333      0.542      0.431
                   car         12         46      0.792      0.261      0.434       0.24
            motorcycle          4          5      0.802        0.8      0.938      0.787
              airplane          5          6      0.794          1      0.948      0.882
                   bus          5          7      0.806      0.714      0.723      0.673
                 train          3          3      0.592          1      0.995      0.886
                 truck          5         12      0.861        0.5      0.581      0.414
                  boat          2          6      0.745       0.49      0.648      0.405
         traffic light          4         14      0.563      0.143      0.206      0.151
             stop sig

## 3. Улучшение бейзлайна

На этом этапе мы попытаемся улучшить результаты бейзлайн-модели.

### 3.a. Формулирование гипотез
1.  **Увеличение количества эпох обучения:** Большее количество эпох может позволить модели лучше настроиться на данные.
2.  **Использование более "тяжелой" модели:** Модели с большим количеством параметров (например, `yolov8s.pt` - small) могут показать лучшее качество за счет большей выразительной способности, но будут обучаться дольше и требовать больше ресурсов.
3.  **Оптимизация гиперпараметров:** Изменение скорости обучения (`lr0`), оптимизатора, или параметров аугментации. Ultralytics YOLO автоматически применяет сильные аугментации, но их можно тонко настроить (например, `degrees`, `translate`, `scale`, `fliplr`, `mosaic`, `mixup`).
4.  **Увеличение размера входного изображения (`imgsz`):** Модели могут лучше распознавать мелкие объекты при большем разрешении, но это увеличивает вычислительную нагрузку.

**Проверяемые гипотезы в рамках данной работы:**
*   **Гипотеза 1:** Увеличение количества эпох обучения (с `baseline_epochs` до `improved_epochs`) улучшит метрики.
*   **Гипотеза 2:** Использование более крупной модели (`yolov8s.pt` вместо `yolov8n.pt`) при том же (или немного увеличенном) количестве эпох улучшит метрики.
*   **Гипотеза 3 (Комбинированная):** Использование более крупной модели (`yolov8s.pt`) И большего числа эпох (`improved_epochs`) И, возможно, других гиперпараметров (например, изменение `lr0` или включение/отключение `mosaic` аугментации) даст наилучший результат.

Для улучшения бейзлайна выберем **Гипотезу 3** как наиболее перспективную: используем модель `yolov8s.pt`, увеличим количество эпох и, возможно, слегка скорректируем некоторые параметры обучения, если потребуется (но для начала оставим параметры по умолчанию, кроме модели и эпох).

### 3.b. Проверка гипотез (в данном случае, сразу формируем улучшенный бейзлайн)

Cогласно выбранной гипотезе, пробуем yolov8s и больше эпох


In [5]:
improved_model_name = 'yolov8s.pt' # Берем модель побольше
improved_epochs = baseline_epochs + 15 # Увеличиваем количество эпох (например, до 40)
improved_imgsz = baseline_imgsz # Оставим тот же размер изображения для сравнения
improved_project_name = 'coco128_improved'
improved_experiment_name = 'yolov8s_improved_run'

# Мы можем также попробовать изменить learning rate или другие параметры
# Например: lr0=0.001 (стандартное 0.01), optimizer='AdamW' (стандартное SGD или Adam)
# Для демонстрации оставим большинство параметров по умолчанию для yolov8s
# Можно добавить/изменить параметры аугментации, например:
# augment_params = {'degrees': 10, 'translate': 0.15, 'scale': 0.6, 'fliplr': 0.5, 'mosaic': 0.5, 'mixup':0.1}
# и передать их в model.train(**augment_params)

print(f"Обучение улучшенной модели: {improved_model_name}")
print(f"Датасет: {dataset_yaml}")
print(f"Количество эпох: {improved_epochs}")
print(f"Размер изображения: {improved_imgsz}")

Обучение улучшенной модели: yolov8s.pt
Датасет: coco128.yaml
Количество эпох: 40
Размер изображения: 640


### 3.c. Формирование улучшенного бейзлайна
Это уже сделано выбором модели и параметров выше.

### 3.d. Обучение модели с улучшенным бейзлайном


In [6]:
model_improved = YOLO(improved_model_name)

try:
    history_improved = model_improved.train(
        data=dataset_yaml,
        epochs=improved_epochs,
        imgsz=improved_imgsz,
        project=improved_project_name,
        name=improved_experiment_name,
        exist_ok=True,
        patience=10 # Можно увеличить patience для более "тяжелой" модели
        # lr0=0.005, # Пример изменения learning rate
        # mosaic=0.8, # Пример изменения параметра аугментации
    )
    print("Обучение улучшенной модели завершено.")
    improved_weights_path = os.path.join(improved_project_name, improved_experiment_name, 'weights', 'best.pt')
    print(f"Лучшие веса сохранены в: {improved_weights_path}")

except Exception as e:
    print(f"Ошибка во время обучения улучшенной модели: {e}")
    history_improved = None
    improved_weights_path = None


100%|██████████| 21.5M/21.5M [00:00<00:00, 107MB/s] 


Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_improved_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

train: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 399.3±86.5 MB/s, size: 52.5 KB)


...

      40/40      5.24G      0.762     0.5949     0.9723        161        640: 100%|██████████| 8/8 [00:02<00:00,  3.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.57it/s]

                   all        128        929       0.92       0.82      0.905      0.753
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 30, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



40 epochs completed in 0.047 hours.
Optimizer stripped from coco128_improved/yolov8s_improved_run/weights/last.pt, 22.6MB
Optimizer stripped from coco128_improved/yolov8s_improved_run/weights/best.pt, 22.6MB

Validating coco128_improved/yolov8s_improved_run/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.48it/s]


                   all        128        929      0.926      0.841      0.913       0.77
                person         61        254       0.97      0.765      0.914      0.735
               bicycle          3          6      0.942      0.833      0.842       0.53
                   car         12         46      0.955      0.463      0.688      0.364
            motorcycle          4          5      0.916          1      0.995      0.904
              airplane          5          6      0.967          1      0.995      0.974
                   bus          5          7      0.939          1      0.995       0.91
                 train          3          3      0.913          1      0.995      0.995
                 truck          5         12      0.901        0.5      0.673      0.484
                  boat          2          6          1      0.713      0.873      0.709
         traffic light          4         14      0.965      0.429      0.465      0.286
             stop sig

### 3.e. Оценка качества модели с улучшенным бейзлайном


In [7]:
if improved_weights_path and os.path.exists(improved_weights_path):
    print(f"Оценка улучшенной модели с весами: {improved_weights_path}")
    model_eval_improved = YOLO(improved_weights_path)

    metrics_improved = model_eval_improved.val(
        data=dataset_yaml,
        imgsz=improved_imgsz,
        split='val',
        project=improved_project_name,
        name=improved_experiment_name + "_eval",
        exist_ok=True
    )

    print("\nРезультаты оценки улучшенной модели:")
    if metrics_improved:
        print(f"  Precision(B): {metrics_improved.box.mp:.4f}")
        print(f"  Recall(B): {metrics_improved.box.mr:.4f}")
        print(f"  mAP50(B): {metrics_improved.box.map50:.4f}")
        print(f"  mAP50-95(B): {metrics_improved.box.map:.4f}")

        results_history['improved'] = {
            'model': improved_model_name.split('.')[0],
            'weights': improved_weights_path,
            'precision': metrics_improved.box.mp,
            'recall': metrics_improved.box.mr,
            'mAP50': metrics_improved.box.map50,
            'mAP50-95': metrics_improved.box.map
        }
    else:
        print("Не удалось получить метрики для улучшенной модели.")
        results_history['improved'] = {
          'model': improved_model_name.split('.')[0],
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
        }
else:
    print("Файл с весами улучшенной модели не найден. Оценка невозможна.")
    results_history['improved'] = {
          'model': improved_model_name.split('.')[0],
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
    }


Оценка улучшенной модели с весами: coco128_improved/yolov8s_improved_run/weights/best.pt
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1529.0±633.8 MB/s, size: 53.4 KB)


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.48it/s]


                   all        128        929      0.923      0.842       0.91      0.767
                person         61        254       0.97      0.768      0.918      0.733
               bicycle          3          6      0.941      0.833      0.841      0.538
                   car         12         46      0.955      0.463      0.678      0.356
            motorcycle          4          5      0.916          1      0.995      0.904
              airplane          5          6      0.967          1      0.995      0.974
                   bus          5          7      0.939          1      0.995      0.898
                 train          3          3      0.913          1      0.995      0.953
                 truck          5         12      0.901        0.5      0.673      0.471
                  boat          2          6      0.904      0.667      0.821      0.593
         traffic light          4         14      0.965      0.429      0.464      0.292
             stop sig

### 3.f. Сравнение результатов моделей с улучшенным бейзлайном в сравнении с результатами из пункта 2


In [8]:
# Вывод таблицы сравнения результатов
print("Сравнение результатов:")
print("--------------------------------------------------------------------------------------------")
print(f"{'Модель':<25} | {'Precision':<10} | {'Recall':<10} | {'mAP@.50':<10} | {'mAP@.50-.95':<12}")
print("--------------------------------------------------------------------------------------------")

if 'baseline' in results_history:
    b = results_history['baseline']
    print(f"{'YOLOv8n (Бейзлайн)':<25} | {b['precision']:.4f}   | {b['recall']:.4f}   | {b['mAP50']:.4f}   | {b['mAP50-95']:.4f}")
else:
    print(f"{'YOLOv8n (Бейзлайн)':<25} | N/A        | N/A        | N/A        | N/A")


if 'improved' in results_history:
    i = results_history['improved']
    print(f"{'YOLOv8s (Улучшенный)':<25} | {i['precision']:.4f}   | {i['recall']:.4f}   | {i['mAP50']:.4f}   | {i['mAP50-95']:.4f}")

    if 'baseline' in results_history and b['mAP50-95'] > 0 : # Проверка, что есть с чем сравнивать
        print("--------------------------------------------------------------------------------------------")
        print("Изменения относительно бейзлайна:")
        precision_diff = (i['precision'] - b['precision']) / b['precision'] * 100 if b['precision'] != 0 else float('inf')
        recall_diff = (i['recall'] - b['recall']) / b['recall'] * 100 if b['recall'] != 0 else float('inf')
        map50_diff = (i['mAP50'] - b['mAP50']) / b['mAP50'] * 100 if b['mAP50'] != 0 else float('inf')
        map_total_diff = (i['mAP50-95'] - b['mAP50-95']) / b['mAP50-95'] * 100 if b['mAP50-95'] != 0 else float('inf')

        print(f"  Δ Precision: {precision_diff:+.2f}%")
        print(f"  Δ Recall: {recall_diff:+.2f}%")
        print(f"  Δ mAP@.50: {map50_diff:+.2f}%")
        print(f"  Δ mAP@.50-.95: {map_total_diff:+.2f}%")

else:
    print(f"{'YOLOv8s (Улучшенный)':<25} | N/A        | N/A        | N/A        | N/A")

print("--------------------------------------------------------------------------------------------")


Сравнение результатов:
--------------------------------------------------------------------------------------------
Модель                    | Precision  | Recall     | mAP@.50    | mAP@.50-.95 
--------------------------------------------------------------------------------------------
YOLOv8n (Бейзлайн)        | 0.7630   | 0.7279   | 0.7858   | 0.6047
YOLOv8s (Улучшенный)      | 0.9231   | 0.8415   | 0.9104   | 0.7671
--------------------------------------------------------------------------------------------
Изменения относительно бейзлайна:
  Δ Precision: +20.99%
  Δ Recall: +15.61%
  Δ mAP@.50: +15.86%
  Δ mAP@.50-.95: +26.86%
--------------------------------------------------------------------------------------------


### 3.g. Выводы по улучшению бейзлайна

По результатам проведенных экспериментов можно сделать следующие выводы:

*   **Исходный бейзлайн (YOLOv8n):** Модель `YOLOv8n` с параметрами обучения (`25` эпох, размер изображения `640x640` – *предполагается на основе типичных настроек из предоставленного ранее ноутбука, уточните если ваши параметры отличались*) показала следующие результаты:
    *   Precision: `0.7630`
    *   Recall: `0.7279`
    *   mAP@0.5: `0.7858`
    *   mAP@0.5:0.95: `0.6047`
    Эти результаты служат отправной точкой для сравнения.

*   **Улучшенный бейзлайн (YOLOv8s):** После применения гипотез по улучшению (использование модели `YOLOv8s`, увеличение количества эпох до `40` – *предполагается на основе типичных настроек `baseline_epochs + 15` из предоставленного ранее ноутбука, уточните если ваши параметры отличались*, при сохранении размера изображения `640x640`), были получены следующие метрики:
    *   Precision: `0.9231`
    *   Recall: `0.8415`
    *   mAP@0.5: `0.9104`
    *   mAP@0.5:0.95: `0.7671`

*   **Сравнение:**
    *   Улучшенная модель показала **значительное улучшение** метрики mAP@0.5:0.95 на `26.86%` по сравнению с бейзлайном.
    *   Метрика mAP@0.5 **улучшилась** на `15.86%`.
    *   Precision **существенно изменилось** c `0.7630` до `0.9231` (рост на `20.99%`), а Recall также **значительно изменилось** c `0.7279` до `0.8415` (рост на `15.61%`).
    *   Гипотеза об использовании более крупной модели (`YOLOv8s` вместо `YOLOv8n`) в сочетании с увеличением числа эпох обучения полностью подтвердилась, что привело к существенному росту всех ключевых метрик качества обнаружения.

*   **Общий вывод:** Предпринятые шаги по улучшению (переход на более крупную архитектуру `YOLOv8s` и увеличение количества эпох обучения) привели к **однозначно положительному** результату, выразившемуся в заметном повышении всех оцениваемых метрик. Это демонстрирует эффективность выбора более мощной модели и достаточного времени для ее обучения на заданном датасете. Для дальнейшего улучшения можно рассмотреть более тонкую настройку гиперпараметров (например, скорости обучения, параметров аугментации), использование еще большего количества эпох или применение более сложных техник аугментации данных.

## 4. Имплементация алгоритма машинного обучения

В контексте Ultralytics YOLO, "самостоятельная имплементация" может означать не создание всей архитектуры с нуля (что является очень объемной задачей), а, например:
1.  Обучение модели YOLO **"с нуля"** (from scratch), используя только YAML-файл конфигурации архитектуры, а не предварительно обученные веса (`.pt` файл). Это позволяет понять процесс обучения без влияния предварительной подготовки на больших датасетах типа COCO.
2.  Создание собственной конфигурации модели (кастомный YAML), изменяя слои, анкеры и т.д. (более сложный вариант).
3.  Реализация кастомного цикла обучения или функции потерь (также продвинутый вариант).

Для данной лабораторной работы мы выберем **первый вариант**: обучим стандартную архитектуру YOLOv8 (например, YOLOv8n) "с нуля", используя ее `.yaml` файл. Это позволит нам сравнить результаты обучения с использованием предобученных весов (fine-tuning) и обучения с случайно инициализированными весами.

### 4.a. "Самостоятельная имплементация" - обучение модели YOLO с нуля (from scratch)

Вместо `yolov8n.pt` (предобученные веса), мы используем `yolov8n.yaml` (определение архитектуры)
Это означает, что модель будет инициализирована случайными весами и обучена только на нашем датасете coco128


In [9]:
 # Задаем параметры для обучения "с нуля"
custom_model_yaml = 'yolov8n.yaml' # Определяет архитектуру, но не веса
# Для обучения с нуля обычно требуется больше эпох, чем для fine-tuning
# Но для демонстрации оставим сопоставимое количество, как у улучшенного бейзлайна
custom_epochs = improved_epochs # Используем то же количество эпох, что и для лучшей модели выше
custom_imgsz = baseline_imgsz # Тот же размер изображения
custom_project_name = 'coco128_custom_scratch'
custom_experiment_name = 'yolov8n_from_scratch_run'

print(f"Обучение 'кастомной' модели (с нуля): {custom_model_yaml}")
print(f"Датасет: {dataset_yaml}")
print(f"Количество эпох: {custom_epochs}")
print(f"Размер изображения: {custom_imgsz}")

Обучение 'кастомной' модели (с нуля): yolov8n.yaml
Датасет: coco128.yaml
Количество эпох: 40
Размер изображения: 640


### 4.b. Обучение имплементированной модели

Создаем модель на основе YAML конфигурации

In [12]:
model_custom_scratch = YOLO(custom_model_yaml) # Загрузка архитектуры

try:
    history_custom_scratch = model_custom_scratch.train(
        data=dataset_yaml,
        epochs=custom_epochs,
        imgsz=custom_imgsz,
        project=custom_project_name,
        name=custom_experiment_name,
        exist_ok=True,
        patience=5 # Установим такое же patience, как в baseline
    )
    print("Обучение 'кастомной' модели (с нуля) завершено.")
    custom_scratch_weights_path = os.path.join(custom_project_name, custom_experiment_name, 'weights', 'best.pt')
    print(f"Лучшие веса сохранены в: {custom_scratch_weights_path}")

except Exception as e:
    print(f"Ошибка во время обучения 'кастомной' модели (с нуля): {e}")
    history_custom_scratch = None
    custom_scratch_weights_path = None


Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_from_scratch_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrain

train: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 385.2±87.7 MB/s, size: 52.5 KB)


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


...

      32/40      2.78G      3.516      5.621      4.142        104        640: 100%|██████████| 8/8 [00:01<00:00,  4.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.89it/s]

                   all        128        929       0.77    0.00156    0.00371   0.000838
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 27, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



32 epochs completed in 0.029 hours.
Optimizer stripped from coco128_custom_scratch/yolov8n_from_scratch_run/weights/last.pt, 6.5MB
Optimizer stripped from coco128_custom_scratch/yolov8n_from_scratch_run/weights/best.pt, 6.5MB

Validating coco128_custom_scratch/yolov8n_from_scratch_run/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.91it/s]


                   all        128        929      0.564    0.00162    0.00593    0.00154
                person         61        254    0.00178    0.00394     0.0034   0.000819
               bicycle          3          6          0          0          0          0
                   car         12         46          0          0          0          0
            motorcycle          4          5          1          0          0          0
              airplane          5          6          1          0          0          0
                   bus          5          7          1          0          0          0
                 train          3          3          1          0          0          0
                 truck          5         12          1          0          0          0
                  boat          2          6          0          0          0          0
         traffic light          4         14          0          0          0          0
             stop sig

### 4.c. Оценка качества имплементированной модели (обученной с нуля)


In [13]:
if custom_scratch_weights_path and os.path.exists(custom_scratch_weights_path):
    print(f"Оценка 'кастомной' модели (с нуля) с весами: {custom_scratch_weights_path}")
    model_eval_custom_scratch = YOLO(custom_scratch_weights_path)

    metrics_custom_scratch = model_eval_custom_scratch.val(
        data=dataset_yaml,
        imgsz=custom_imgsz,
        split='val',
        project=custom_project_name,
        name=custom_experiment_name + "_eval",
        exist_ok=True
    )

    print("\nРезультаты оценки 'кастомной' модели (обученной с нуля):")
    if metrics_custom_scratch:
        print(f"  Precision(B): {metrics_custom_scratch.box.mp:.4f}")
        print(f"  Recall(B): {metrics_custom_scratch.box.mr:.4f}")
        print(f"  mAP50(B): {metrics_custom_scratch.box.map50:.4f}")
        print(f"  mAP50-95(B): {metrics_custom_scratch.box.map:.4f}")

        results_history['custom_scratch'] = {
            'model': custom_model_yaml.split('.')[0] + " (scratch)",
            'weights': custom_scratch_weights_path,
            'precision': metrics_custom_scratch.box.mp,
            'recall': metrics_custom_scratch.box.mr,
            'mAP50': metrics_custom_scratch.box.map50,
            'mAP50-95': metrics_custom_scratch.box.map
        }
    else:
        print("Не удалось получить метрики для 'кастомной' модели (с нуля).")
        results_history['custom_scratch'] = {
          'model': custom_model_yaml.split('.')[0] + " (scratch)",
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
        }
else:
    print("Файл с весами 'кастомной' модели (с нуля) не найден. Оценка невозможна.")
    results_history['custom_scratch'] = {
          'model': custom_model_yaml.split('.')[0] + " (scratch)",
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
        }

Оценка 'кастомной' модели (с нуля) с весами: coco128_custom_scratch/yolov8n_from_scratch_run/weights/best.pt
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1618.9±846.2 MB/s, size: 53.4 KB)


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.17it/s]


                   all        128        929      0.482    0.00471    0.00468    0.00102
                person         61        254    0.00629     0.0236    0.00272   0.000669
               bicycle          3          6          0          0          0          0
                   car         12         46          0          0          0          0
            motorcycle          4          5          0          0          0          0
              airplane          5          6          1          0          0          0
                   bus          5          7          1          0          0          0
                 train          3          3          0          0          0          0
                 truck          5         12          1          0          0          0
                  boat          2          6          0          0          0          0
         traffic light          4         14          0          0          0          0
             stop sig

### 4.d. Сравнение результатов имплементированных моделей (обученных с нуля) в сравнении с результатами из пункта 2 (бейзлайн)


In [14]:
print("Сравнение результатов: Бейзлайн vs. Обучение с нуля")
print("--------------------------------------------------------------------------------------------")
print(f"{'Модель':<30} | {'Precision':<10} | {'Recall':<10} | {'mAP@.50':<10} | {'mAP@.50-.95':<12}")
print("--------------------------------------------------------------------------------------------")

if 'baseline' in results_history:
    b = results_history['baseline']
    print(f"{'YOLOv8n (Бейзлайн, fine-tune)':<30} | {b['precision']:.4f}   | {b['recall']:.4f}   | {b['mAP50']:.4f}   | {b['mAP50-95']:.4f}")
else:
    print(f"{'YOLOv8n (Бейзлайн, fine-tune)':<30} | N/A        | N/A        | N/A        | N/A")

if 'custom_scratch' in results_history:
    cs = results_history['custom_scratch']
    print(f"{'YOLOv8n (Кастом, с нуля)':<30} | {cs['precision']:.4f}   | {cs['recall']:.4f}   | {cs['mAP50']:.4f}   | {cs['mAP50-95']:.4f}")

    if 'baseline' in results_history and b['mAP50-95'] > 0 :
        print("--------------------------------------------------------------------------------------------")
        print("Изменения 'с нуля' относительно бейзлайна (fine-tune):")
        precision_diff_cs = (cs['precision'] - b['precision']) / b['precision'] * 100 if b['precision'] != 0 else float('inf')
        recall_diff_cs = (cs['recall'] - b['recall']) / b['recall'] * 100 if b['recall'] != 0 else float('inf')
        map50_diff_cs = (cs['mAP50'] - b['mAP50']) / b['mAP50'] * 100 if b['mAP50'] != 0 else float('inf')
        map_total_diff_cs = (cs['mAP50-95'] - b['mAP50-95']) / b['mAP50-95'] * 100 if b['mAP50-95'] != 0 else float('inf')

        print(f"  Δ Precision: {precision_diff_cs:+.2f}%")
        print(f"  Δ Recall: {recall_diff_cs:+.2f}%")
        print(f"  Δ mAP@.50: {map50_diff_cs:+.2f}%")
        print(f"  Δ mAP@.50-.95: {map_total_diff_cs:+.2f}%")
else:
    print(f"{'YOLOv8n (Кастом, с нуля)':<30} | N/A        | N/A        | N/A        | N/A")

print("--------------------------------------------------------------------------------------------")


Сравнение результатов: Бейзлайн vs. Обучение с нуля
--------------------------------------------------------------------------------------------
Модель                         | Precision  | Recall     | mAP@.50    | mAP@.50-.95 
--------------------------------------------------------------------------------------------
YOLOv8n (Бейзлайн, fine-tune)  | 0.7630   | 0.7279   | 0.7858   | 0.6047
YOLOv8n (Кастом, с нуля)       | 0.4821   | 0.0047   | 0.0047   | 0.0010
--------------------------------------------------------------------------------------------
Изменения 'с нуля' относительно бейзлайна (fine-tune):
  Δ Precision: -36.81%
  Δ Recall: -99.35%
  Δ mAP@.50: -99.40%
  Δ mAP@.50-.95: -99.83%
--------------------------------------------------------------------------------------------


### 4.e. Выводы по сравнению "имплементированной" модели (с нуля) с бейзлайном (fine-tune)

*   **Бейзлайн (fine-tune `yolov8n.pt`):** Модель, дообученная на основе предварительно обученных на COCO весов, показала mAP@0.5:0.95 = `0.6047`.
*   **"Кастомная" модель (train from scratch `yolov8n.yaml`):** Модель той же архитектуры (`YOLOv8n`), но обученная с нуля только на датасете COCO128 (предположительно, с тем же количеством эпох, что и улучшенный бейзлайн, например, `40` эпох – *уточните, если число эпох отличалось*), показала mAP@0.5:0.95 = `0.0010`.

*   **Сравнение:**
    *   Обучение с нуля привело к **катастрофическому ухудшению** качества по сравнению с дообучением (fine-tuning) предварительно обученной модели. Разница в mAP@0.5:0.95 составила `-99.83%` (падение с 0.6047 до 0.0010).
    *   Особенно показательно падение метрики Recall (`-99.35%` с `0.7279` до `0.0047`), что свидетельствует о почти полной неспособности модели, обученной с нуля на таком малом датасете, обнаруживать объекты. Метрика mAP@0.5 также упала на `99.40%`.
    *   Это **ожидаемый результат**, поскольку COCO128 является очень маленьким датасетом. Предварительное обучение на большом и разнообразном наборе данных (полный COCO) дает модели хорошее начальное представление о признаках объектов, что позволяет ей быстрее и качественнее адаптироваться к целевому датасету при дообучении. Обучение с нуля на COCO128, даже с достаточным количеством эпох для fine-tuning более крупной модели, не позволяет модели выучить сколь-нибудь значимые представления объектов.
    *   Результаты не являются неожиданными и полностью соответствуют теории transfer learning.

*   **Общий вывод:** Дообучение (fine-tuning) предобученных моделей является **однозначно предпочтительной** стратегией при работе с небольшими целевыми наборами данных, так как позволяет достичь несравненно лучших результатов по сравнению с обучением "с нуля". Для обучения "с нуля" на малых датасетах, таких как COCO128, даже при сопоставимом или увеличенном количестве эпох, модель не способна эффективно обучиться, что демонстрируют полученные крайне низкие метрики. Это подчеркивает критическую важность использования знаний, полученных на больших датасетах.

### 4.f. Добавление техник из улучшенного бейзлайна (пункт 3c) к "кастомной" модели
В пункте 3с мы определили улучшенный бейзлайн, используя модель `yolov8s.pt` (более крупная архитектура) и увеличенное количество эпох. Попробуем применить аналогичный подход к обучению "с нуля": будем использовать архитектуру `yolov8s.yaml` и такое же количество эпох, как в пункте 3d.

Это означает, что мы будем обучать более крупную модель **с нуля**.

### 4.g. Обучение "кастомной" модели (yolov8s с нуля) с техниками из улучшенного бейзлайна



In [15]:
# Параметры для обучения "кастомной улучшенной" модели (yolov8s.yaml с нуля)
custom_imp_model_yaml = 'yolov8s.yaml' # Более крупная архитектура
custom_imp_epochs = improved_epochs # То же количество эпох, что и в п.3.d
custom_imp_imgsz = improved_imgsz # Тот же размер изображения
custom_imp_project_name = 'coco128_custom_imp_scratch'
custom_imp_experiment_name = 'yolov8s_from_scratch_imp_run'

# Другие параметры (аугментации, lr0 и т.д.) из улучшенного бейзлайна пункта 3
# также могут быть применены здесь, если они были явно заданы там.
# В нашем примере из п.3 мы в основном меняли модель и эпохи.
# model_improved.train(... mosaic=0.8 ...)

print(f"Обучение 'кастомной улучшенной' модели (с нуля): {custom_imp_model_yaml}")
print(f"Датасет: {dataset_yaml}")
print(f"Количество эпох: {custom_imp_epochs}")
print(f"Размер изображения: {custom_imp_imgsz}")

# Создаем модель
model_custom_imp_scratch = YOLO(custom_imp_model_yaml)

try:
    history_custom_imp_scratch = model_custom_imp_scratch.train(
        data=dataset_yaml,
        epochs=custom_imp_epochs,
        imgsz=custom_imp_imgsz,
        project=custom_imp_project_name,
        name=custom_imp_experiment_name,
        exist_ok=True,
        patience=10 # Можно оставить или адаптировать
        # mosaic=0.8, # Если такой параметр был в улучшенном бейзлайне п.3
    )
    print("Обучение 'кастомной улучшенной' модели (с нуля) завершено.")
    custom_imp_scratch_weights_path = os.path.join(custom_imp_project_name, custom_imp_experiment_name, 'weights', 'best.pt')
    print(f"Лучшие веса сохранены в: {custom_imp_scratch_weights_path}")

except Exception as e:
    print(f"Ошибка во время обучения 'кастомной улучшенной' модели (с нуля): {e}")
    history_custom_imp_scratch = None
    custom_imp_scratch_weights_path = None


Обучение 'кастомной улучшенной' модели (с нуля): yolov8s.yaml
Датасет: coco128.yaml
Количество эпох: 40
Размер изображения: 640
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_from_scratch_imp_run, nbs=64, nm

train: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 385.3±144.8 MB/s, size: 52.5 KB)


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


...

      34/40      5.49G      3.489      5.421      4.033        100        640: 100%|██████████| 8/8 [00:02<00:00,  3.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.60it/s]

                   all        128        929      0.355    0.00438    0.00245   0.000734
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 24, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



34 epochs completed in 0.050 hours.
Optimizer stripped from coco128_custom_imp_scratch/yolov8s_from_scratch_imp_run/weights/last.pt, 22.6MB
Optimizer stripped from coco128_custom_imp_scratch/yolov8s_from_scratch_imp_run/weights/best.pt, 22.6MB

Validating coco128_custom_imp_scratch/yolov8s_from_scratch_imp_run/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.60it/s]


                   all        128        929     0.0098      0.017     0.0104    0.00229
                person         61        254    0.00353       0.15    0.00242     0.0009
               bicycle          3          6          0          0          0          0
                   car         12         46          0          0          0          0
            motorcycle          4          5          0          0          0          0
              airplane          5          6          0          0          0          0
                   bus          5          7        0.5      0.143      0.286     0.0857
                 train          3          3          0          0          0          0
                 truck          5         12          0          0          0          0
                  boat          2          6          0          0          0          0
         traffic light          4         14          0          0          0          0
             stop sig

### 4.h. Оценка качества "кастомной улучшенной" модели (yolov8s с нуля)


In [16]:
if custom_imp_scratch_weights_path and os.path.exists(custom_imp_scratch_weights_path):
    print(f"Оценка 'кастомной улучшенной' модели (с нуля) с весами: {custom_imp_scratch_weights_path}")
    model_eval_custom_imp_scratch = YOLO(custom_imp_scratch_weights_path)

    metrics_custom_imp_scratch = model_eval_custom_imp_scratch.val(
        data=dataset_yaml,
        imgsz=custom_imp_imgsz,
        split='val',
        project=custom_imp_project_name,
        name=custom_imp_experiment_name + "_eval",
        exist_ok=True
    )

    print("\nРезультаты оценки 'кастомной улучшенной' модели (обученной с нуля):")
    if metrics_custom_imp_scratch:
        print(f"  Precision(B): {metrics_custom_imp_scratch.box.mp:.4f}")
        print(f"  Recall(B): {metrics_custom_imp_scratch.box.mr:.4f}")
        print(f"  mAP50(B): {metrics_custom_imp_scratch.box.map50:.4f}")
        print(f"  mAP50-95(B): {metrics_custom_imp_scratch.box.map:.4f}")

        results_history['custom_imp_scratch'] = {
            'model': custom_imp_model_yaml.split('.')[0] + " (scratch, imp)",
            'weights': custom_imp_scratch_weights_path,
            'precision': metrics_custom_imp_scratch.box.mp,
            'recall': metrics_custom_imp_scratch.box.mr,
            'mAP50': metrics_custom_imp_scratch.box.map50,
            'mAP50-95': metrics_custom_imp_scratch.box.map
        }
    else:
        print("Не удалось получить метрики для 'кастомной улучшенной' модели (с нуля).")
        results_history['custom_imp_scratch'] = {
          'model': custom_imp_model_yaml.split('.')[0] + " (scratch, imp)",
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
        }
else:
    print("Файл с весами 'кастомной улучшенной' модели (с нуля) не найден. Оценка невозможна.")
    results_history['custom_imp_scratch'] = {
          'model': custom_imp_model_yaml.split('.')[0] + " (scratch, imp)",
          'weights': "N/A",
          'precision': 0, 'recall': 0, 'mAP50': 0, 'mAP50-95': 0
        }


Оценка 'кастомной улучшенной' модели (с нуля) с весами: coco128_custom_imp_scratch/yolov8s_from_scratch_imp_run/weights/best.pt
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1624.2±294.9 MB/s, size: 53.4 KB)


val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.08it/s]


                   all        128        929    0.00276     0.0149    0.00842    0.00176
                person         61        254    0.00353      0.146    0.00239   0.000861
               bicycle          3          6          0          0          0          0
                   car         12         46          0          0          0          0
            motorcycle          4          5          0          0          0          0
              airplane          5          6          0          0          0          0
                   bus          5          7          0          0          0          0
                 train          3          3          0          0          0          0
                 truck          5         12          0          0          0          0
                  boat          2          6          0          0          0          0
         traffic light          4         14          0          0          0          0
             stop sig

### 4.i. Сравнение результатов "кастомной улучшенной" модели (yolov8s с нуля) с результатами из пункта 3 (улучшенный бейзлайн, yolov8s fine-tune)


In [17]:
print("Сравнение результатов: Улучшенный Бейзлайн (fine-tune) vs. 'Кастомная Улучшенная' (scratch)")
print("----------------------------------------------------------------------------------------------------")
print(f"{'Модель':<40} | {'Precision':<10} | {'Recall':<10} | {'mAP@.50':<10} | {'mAP@.50-.95':<12}")
print("----------------------------------------------------------------------------------------------------")

if 'improved' in results_history: # Это улучшенный бейзлайн из п.3 (yolov8s.pt fine-tune)
    i = results_history['improved']
    print(f"{'YOLOv8s (Улучшенный, fine-tune)':<40} | {i['precision']:.4f}   | {i['recall']:.4f}   | {i['mAP50']:.4f}   | {i['mAP50-95']:.4f}")
else:
    print(f"{'YOLOv8s (Улучшенный, fine-tune)':<40} | N/A        | N/A        | N/A        | N/A")

if 'custom_imp_scratch' in results_history: # Это yolov8s.yaml обученный с нуля с "улучшениями"
    cis = results_history['custom_imp_scratch']
    print(f"{'YOLOv8s (Кастом-Улучш, с нуля)':<40} | {cis['precision']:.4f}   | {cis['recall']:.4f}   | {cis['mAP50']:.4f}   | {cis['mAP50-95']:.4f}")

    if 'improved' in results_history and i['mAP50-95'] > 0:
        print("----------------------------------------------------------------------------------------------------")
        print("Изменения 'кастом-улучш, с нуля' относительно 'улучшенного fine-tune':")
        precision_diff_cis_vs_i = (cis['precision'] - i['precision']) / i['precision'] * 100 if i['precision'] !=0 else float('inf')
        recall_diff_cis_vs_i = (cis['recall'] - i['recall']) / i['recall'] * 100 if i['recall'] !=0 else float('inf')
        map50_diff_cis_vs_i = (cis['mAP50'] - i['mAP50']) / i['mAP50'] * 100 if i['mAP50'] !=0 else float('inf')
        map_total_diff_cis_vs_i = (cis['mAP50-95'] - i['mAP50-95']) / i['mAP50-95'] * 100 if i['mAP50-95'] !=0 else float('inf')

        print(f"  Δ Precision: {precision_diff_cis_vs_i:+.2f}%")
        print(f"  Δ Recall: {recall_diff_cis_vs_i:+.2f}%")
        print(f"  Δ mAP@.50: {map50_diff_cis_vs_i:+.2f}%")
        print(f"  Δ mAP@.50-.95: {map_total_diff_cis_vs_i:+.2f}%")

else:
    print(f"{'YOLOv8s (Кастом-Улучш, с нуля)':<40} | N/A        | N/A        | N/A        | N/A")

print("----------------------------------------------------------------------------------------------------")


Сравнение результатов: Улучшенный Бейзлайн (fine-tune) vs. 'Кастомная Улучшенная' (scratch)
----------------------------------------------------------------------------------------------------
Модель                                   | Precision  | Recall     | mAP@.50    | mAP@.50-.95 
----------------------------------------------------------------------------------------------------
YOLOv8s (Улучшенный, fine-tune)          | 0.9231   | 0.8415   | 0.9104   | 0.7671
YOLOv8s (Кастом-Улучш, с нуля)           | 0.0028   | 0.0149   | 0.0084   | 0.0018
----------------------------------------------------------------------------------------------------
Изменения 'кастом-улучш, с нуля' относительно 'улучшенного fine-tune':
  Δ Precision: -99.70%
  Δ Recall: -98.23%
  Δ mAP@.50: -99.07%
  Δ mAP@.50-.95: -99.77%
----------------------------------------------------------------------------------------------------


### 4.j. Выводы по сравнению "кастомной улучшенной" модели (с нуля) с улучшенным бейзлайном (fine-tune)

*   **Улучшенный бейзлайн (fine-tune `yolov8s.pt`):** Модель `yolov8s`, дообученная на COCO128 с `40` эпохами (*предполагается на основе предыдущих данных, уточните, если ваше M отличалось*), показала mAP@0.5:0.95 = `0.7671`.
*   **"Кастомная улучшенная" модель (train `yolov8s.yaml` from scratch):** Модель архитектуры `yolov8s`, обученная с нуля на COCO128 с `40` эпохами (*те же параметры, что и для улучшенного бейзлайна*), показала mAP@0.5:0.95 = `0.0018`.

*   **Сравнение:**
    *   Даже при использовании более крупной архитектуры (`yolov8s.yaml`) и тех же "улучшенных" параметров обучения (количество эпох), обучение с нуля все еще **катастрофически уступает** дообучению (fine-tuning) модели `yolov8s.pt`. Разница в mAP@0.5:0.95 составила `-99.77%` (падение с `0.7671` до `0.0018`).
    *   Все остальные метрики (Precision, Recall, mAP@0.5) также показали крайне значительное падение (`-99.70%`, `-98.23%`, `-99.07%` соответственно).
    *   Это **полностью подтверждает** предыдущий вывод о критической важности предварительного обучения, особенно для небольших датасетов. Увеличение размера архитектуры при обучении с нуля на малом наборе данных не компенсирует отсутствие знаний, полученных при pre-training на большом датасете. Модель без предварительного обучения не смогла извлечь полезные признаки даже с более сложной архитектурой.

*   **Общий вывод:** Для небольших датасетов, таких как COCO128, использование предварительно обученных весов (fine-tuning) остается наиболее эффективной стратегией для достижения высокого качества моделей обнаружения объектов. Обучение "с нуля", даже с более крупными архитектурами и увеличенным числом эпох, **категорически не позволяет** достичь сколько-нибудь приемлемого уровня производительности без значительного увеличения объема данных или продолжительности обучения. Эксперименты с обучением "с нуля" наглядно демонстрируют ценность transfer learning и сложность обучения глубоких моделей на ограниченных данных.

---

## Общие выводы по лабораторной работе №8

В ходе выполнения лабораторной работы были проведены исследования с моделями обнаружения объектов семейства YOLO (на примере Ultralytics YOLOv8) на датасете COCO128.

1.  **Бейзлайн:** Была обучена базовая модель (`yolov8n.pt` fine-tune) с `25` эпохами (*N, уточните, если ваше значение отличалось*), которая показала результат mAP@0.5:0.95 = `0.6047`.
2.  **Улучшение бейзлайна:** Путем использования более крупной модели `yolov8s.pt` и увеличения числа эпох до `40` (*M, уточните, если ваше значение отличалось*) удалось **значительно улучшить** качество, достигнув mAP@0.5:0.95 = `0.7671`, что на `+26.86%` **лучше** бейзлайна. Это подтвердило гипотезу о том, что выбор более сложной архитектуры и увеличение продолжительности обучения могут положительно сказаться на результате при дообучении.
3.  **"Кастомная" имплементация (обучение с нуля):**
    *   Обучение модели `yolov8n.yaml` с нуля показало mAP@0.5:0.95 = `0.0010`, что значительно (на `99.83%`) **хуже** чем fine-tuning (`yolov8n.pt`). Это подчеркивает критическую важность transfer learning для малых датасетов.
    *   Обучение более крупной модели `yolov8s.yaml` с нуля с "улучшенными" параметрами (аналогично п.2) дало mAP@0.5:0.95 = `0.0018`. Этот результат **все еще катастрофически уступал** (на `99.77%`) fine-tuning модели `yolov8s.pt` (`0.7671`), что **дополнительно и неопровержимо подтверждает** выводы о подавляющем преимуществе fine-tuning на малых датасетах, даже при использовании более мощных архитектур для обучения с нуля.

**Ключевые наблюдения:**
*   **Transfer Learning (Fine-tuning):** Является абсолютно доминирующим и мощным инструментом, особенно при ограниченном объеме целевых данных как COCO128. Использование предобученных весов (`.pt` файлы) дает несравнимо лучший старт и приводит к на порядки более высоким результатам по сравнению с обучением с нуля.
*   **Размер модели и время обучения (при fine-tuning):** Более крупные модели (как `yolov8s` по сравнению с `yolov8n`) в сочетании с достаточным количеством эпох для дообучения могут дать значительный прирост качества.
*   **Обучение с нуля на малых датасетах:** Для сложных задач, таких как обнаружение объектов, и современных глубоких архитектур, обучение с нуля на датасетах размером с COCO128 практически не приводит к формированию работоспособной модели, независимо от размера архитектуры (в разумных пределах) или количества эпох, сопоставимых с fine-tuning.
*   **Датасет:** COCO128 удобен для быстрых экспериментов и демонстрации основных принципов, но его малый размер делает результаты обучения с нуля нерепрезентативными для реальных задач, где такие модели не смогли бы выучить общие признаки.

**Практическая значимость:** Данная работа наглядно демонстрирует стандартный пайплайн работы с моделями обнаружения объектов и критическую важность стратегии transfer learning (fine-tuning) при работе с ограниченными наборами данных. Понимание того, что попытки обучить сложные модели "с нуля" на малых данных скорее всего обречены на неудачу, экономит время и ресурсы, направляя усилия на правильную адаптацию существующих мощных предобученных моделей.